In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
from itertools import product
from tqdm.notebook import tqdm

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix  

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
df = pd.read_csv('data/responses_.csv')

In [ ]:
df.describe()

In [ ]:
df.isna().sum().sum()

In [ ]:
df.columns

# Get Average Per Class

In [ ]:
big_list = []
small_list = []
for i in list(df.columns):
    if small_list == []:
        small_list.append(i)
    if i[:-1] == small_list[0][:-1]:
        small_list.append(i)
    if i[:-1] != small_list[0][:-1]:
        big_list.append(list(set(small_list)))
        small_list = []
        small_list.append(i)

In [ ]:
big_list.append(['FI1', 'FI2', 'FI3', 'FI4'])
big_list

In [ ]:
median_df = pd.DataFrame()

for small_list in big_list:
    marker = small_list[0][:-1]
    ave = df[small_list].median(axis=1).values
    ceil = ave # np.ceil(ave)
    median_df[marker] = ceil

In [ ]:
median_df.to_csv('data/median_df.csv', index=False) #Save Median

In [ ]:
median_df.shape

In [ ]:
median_df

# Perform Chi-Square Test

In [ ]:
cat_var1  = list(median_df.columns)
cat_var2  = list(median_df.columns)

In [ ]:
# Get All Possible Combinations

cat_var_prod = list(product(cat_var1,cat_var2, repeat = 1))

In [ ]:
result = []

for i in tqdm(cat_var_prod):
    if i[0] != i[1]:
        result.append((i[0],i[1],list(ss.chi2_contingency(pd.crosstab(
            median_df[i[0]], median_df[i[1]])))[1]))

In [ ]:
chi_test_output = pd.DataFrame(result, columns = ['var1', 'var2', 'coeff'])
chi_test_output['coeff'] = np.around(chi_test_output['coeff'].values, decimals=3)
pivot_ct = chi_test_output.pivot(index='var1', columns='var2', values='coeff')

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(pivot_ct, annot=True, cmap='crest')

plt.show()